<a href="https://colab.research.google.com/github/dani-lbnl/mudit/blob/main/Elsevier_Extract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Elsevier

In [10]:
import requests 
from io import BytesIO
import re 
import os

In [8]:
!pip install elsapy

In [11]:
from elsapy.elsclient import ElsClient
from elsapy.elsprofile import ElsAuthor, ElsAffil
from elsapy.elsdoc import FullDoc, AbsDoc
from elsapy.elssearch import ElsSearch
import json

In [12]:
# create config.json
dict_api =    {
	    "apikey": "856d163d12a1f149d801ee44e166857e",
	    "insttoken": ""
    }

api_json = json.dumps(dict_api)
# print(api_json)  


my_file = r'config.json'
with open(my_file, 'w', encoding='utf-8') as file:
    file.write(api_json)


## Load configuration
con_file = open("config.json")
config = json.load(con_file)
con_file.close()

## Initialize client
client = ElsClient(config['apikey'])
client.inst_token = config['insttoken']

In [5]:
#   retrieving all results
doc_srch = ElsSearch("rechargeable lithium metal",'sciencedirect')
doc_srch.execute(client, get_all = False)
print ("doc_srch has", len(doc_srch.results), "results.")

HTTPError: ignored

In [ ]:
#   retrieving all results
doc_srch = ElsSearch("rechargeable lithium metal",'scopus')
doc_srch.execute(client, get_all = True)
print ("doc_srch has", len(doc_srch.results), "results.")

In [ ]:
!pip install pickle

In [ ]:
import pickle
drive.mount('/content/drive/MyDrive/LBNL')
DATA_PATH = "/content/drive/MyDrive/LBNL"
infile = open(DATA_PATH+'/notMNIST.pickle','rb')
best_model2 = pickle.load(infile)


In [ ]:
# get scopus ID
scopus_id = []
for p in doc_srch.results:
  if p['openaccess']==1:
    scopus_id.append(p['dc:identifier'])

In [14]:
# get doi
doi = []
for p in doc_srch.results:
  if p['openaccess']=='1':
    doi.append(p['prism:doi'])

In [9]:
doc_srch.results[9]

{'@_fa': 'true',
 'affiliation': [{'@_fa': 'true',
   'affiliation-city': 'Thuwal',
   'affiliation-country': 'Saudi Arabia',
   'affilname': 'King Abdullah University of Science and Technology'}],
 'article-number': '14',
 'citedby-count': '0',
 'dc:creator': 'Deokar G.',
 'dc:identifier': 'SCOPUS_ID:85126090467',
 'dc:title': 'Chemical vapor deposition-grown nitrogen-doped graphene’s synthesis, characterization and applications',
 'eid': '2-s2.0-85126090467',
 'freetoread': {'value': [{'$': 'all'},
   {'$': 'publisherfullgold'},
   {'$': 'repository'},
   {'$': 'repositoryvor'}]},
 'freetoreadLabel': {'value': [{'$': 'All Open Access'},
   {'$': 'Gold'},
   {'$': 'Green'}]},
 'link': [{'@_fa': 'true',
   '@href': 'https://api.elsevier.com/content/abstract/scopus_id/85126090467',
   '@ref': 'self'},
  {'@_fa': 'true',
   '@href': 'https://api.elsevier.com/content/abstract/scopus_id/85126090467?field=author,affiliation',
   '@ref': 'author-affiliation'},
  {'@_fa': 'true',
   '@href': 

In [15]:
len(doi)

25

In [20]:
"""
Elsevier web-scraper. PLEASE use your own API key.
"""

# __author__ = "Shu Huang"
# __email__ = "sh2009@cam.ac.uk"

import json
import requests
import numpy as np

API_KEY = "856d163d12a1f149d801ee44e166857e"
query = "rechargeable lithium metal"
base_url = 'https://api.elsevier.com/content/search/scopus'

data = {"qs": query,
        "date": 2019,
        # "volume": 0,
        "display": {
            "show": 10,
            "offset": 0
        }}

headers = {'x-els-apikey': API_KEY,
           'Content-Type': 'application/json',
           'Accept': 'application/json'}


def get_response(url, data, headers):
    """
    Return the response from Elsevier
    :param url: <str> base_url
    :param data: <dict> data parameters
    :param headers: <dict> headers
    :return: response
    """
    response = requests.put(url, data=json.dumps(data), headers=headers)
    response = response.text.replace('false', 'False').replace('true', 'True')
    try:
        response = eval(response)
    except BaseException:
        print(response)
    return response


def get_doi(data, volume, year):
    """
    Get DOIs
    :param data: <dict> data parameters
    :param volume: <int> the volume index
    :param year: <int> the year index
    :return: <list> of <str> list of DOIs
    """
    dois = []
    data['volume'] = volume
    data["date"] = year
    response = get_response(base_url, data, headers)
    if 'resultsFound' in response.keys():
        n = int(np.ceil(response['resultsFound'] / 100))
    else:
        n = 60
    for offset in range(n + 1):
        data["display"]["offset"] = offset
        response = get_response(base_url, data, headers)
        if 'results' in response.keys():
            results = response['results']
            for result in results:
                if 'doi' in result:
                    dois.append(result['doi'])
    return dois


# def download_doi(doi):
#     """
#     Download the paper according to the DOI
#     :param doi: <str> DOI
#     """
#     with open(str(doi) + '.xml', 'w', encoding='utf-8') as f:
#         request_url = 'https://api.elsevier.com/content/article/scopus_id/{}?apiKey={}&httpAccept=text%2Fxml'.format(
#             doi, API_KEY)
#         text = requests.get(request_url).text
#         f.write(text)
#     return

In [ ]:
def download_scopus(doi):
    """
    Download the paper according to the DOI
    :param doi: <str> DOI
    """
    with open(str(doi) + '.xml', 'w', encoding='utf-8') as f:
        request_url = 'https://api.elsevier.com/content/article/scopus_id/{}?apiKey={}&httpAccept=text%2Fxml'.format(
            doi, API_KEY)
        print(request_url)
        text = requests.get(request_url).text
        f.write(text)
    return

In [22]:
data

{'date': 2019,
 'display': {'offset': 0, 'show': 10},
 'qs': 'rechargeable lithium metal'}

In [24]:
get_doi(data,1,2019)

[]

In [17]:
len(doi)

25

In [18]:
num_list = []
for i in range(1,26):
  num_list.append(i)


In [11]:
def download_doi(doi):
    """
    Download the paper according to the DOI
    :param doi: <str> DOI
    """
    with open("doi/" + str(doi) + '.xml', 'w', encoding='utf-8') as f:
        request_url = 'https://api.elsevier.com/content/article/doi/{}?apiKey={}&httpAccept=text%2Fxml'.format(
            doi, API_KEY)
        print(request_url)
        text = requests.get(request_url).text
        f.write(text)
    return

In [21]:
'https://api.elsevier.com/content/article/doi/10.1038/s41467-022-28988-0?apiKey=856d163d12a1f149d801ee44e166857e&httpAccept=text%2Fxml'

'https://api.elsevier.com/content/article/doi/10.1038/s41467-022-28988-0?apiKey=856d163d12a1f149d801ee44e166857e&httpAccept=text%2Fxml'

In [12]:
 request_url = 'https://api.elsevier.com/content/article/scopus_id/{}?apiKey={}'.format(
            '85122632679', API_KEY)
 
 print(request_url)
 # getting the error - "Resource not found"

https://api.elsevier.com/content/article/scopus_id/85122632679?apiKey=856d163d12a1f149d801ee44e166857e


In [14]:
 request_url = 'https://api.elsevier.com/content/article/doi/{}?apiKey={}&httpAccept=text%2Fxml'.format(
            '10.1038/s41598-022-08691-2', 	'856d163d12a1f149d801ee44e166857e')
 
 print(request_url)
 # getting the error - "Resource not found"

https://api.elsevier.com/content/article/doi/10.1038/s41598-022-08691-2?apiKey=856d163d12a1f149d801ee44e166857e&httpAccept=text%2Fxml


In [20]:
doi

['10.1038/s41467-022-28963-9',
 '10.1038/s41467-022-29118-6',
 '10.1038/s41598-022-08691-2',
 '10.1038/s41467-022-29022-z',
 '10.1038/s41467-022-28988-0',
 '10.1186/s40659-022-00382-6',
 '10.1038/s41467-022-28959-5',
 '10.1038/s41528-022-00147-x',
 '10.1038/s41467-022-28880-x',
 '10.1038/s41699-022-00287-8',
 '10.1038/s41598-022-07853-6',
 '10.1038/s41598-022-07858-1',
 '10.1038/s41467-022-28792-w',
 '10.1038/s41467-022-28793-9',
 '10.1007/s40820-022-00793-w',
 '10.1038/s41598-022-07036-3',
 '10.1007/s40820-022-00806-8',
 '10.1038/s42004-022-00626-2',
 '10.1038/s41528-022-00144-0',
 '10.1038/s42004-022-00628-0',
 '10.1038/s41467-022-28380-y',
 '10.1038/s41467-022-28325-5',
 '10.1038/s41524-021-00693-4',
 '10.1038/s41467-022-28381-x',
 '10.1038/s41467-022-28176-0']

In [15]:
for id in doi:
  print(id)
  download_doi(id)

10.1038/s41467-022-28963-9


FileNotFoundError: ignored

In [ ]:
for id in scopus_id:
  print(id)
  download_scopus(id)

SCOPUS_ID:85126720450
https://api.elsevier.com/content/article/scopus_id/SCOPUS_ID:85126720450?apiKey=856d163d12a1f149d801ee44e166857e&httpAccept=text%2Fxml
SCOPUS_ID:85126558210
https://api.elsevier.com/content/article/scopus_id/SCOPUS_ID:85126558210?apiKey=856d163d12a1f149d801ee44e166857e&httpAccept=text%2Fxml
SCOPUS_ID:85126531853
https://api.elsevier.com/content/article/scopus_id/SCOPUS_ID:85126531853?apiKey=856d163d12a1f149d801ee44e166857e&httpAccept=text%2Fxml
SCOPUS_ID:85126269582
https://api.elsevier.com/content/article/scopus_id/SCOPUS_ID:85126269582?apiKey=856d163d12a1f149d801ee44e166857e&httpAccept=text%2Fxml
SCOPUS_ID:85126250517
https://api.elsevier.com/content/article/scopus_id/SCOPUS_ID:85126250517?apiKey=856d163d12a1f149d801ee44e166857e&httpAccept=text%2Fxml
SCOPUS_ID:85126237860
https://api.elsevier.com/content/article/scopus_id/SCOPUS_ID:85126237860?apiKey=856d163d12a1f149d801ee44e166857e&httpAccept=text%2Fxml
SCOPUS_ID:85126224116
https://api.elsevier.com/content/art

## search by SCOPUS ID

In [17]:
num_list = []
for i in range(1,26):
  num_list.append(i)

In [16]:
doi

['10.1038/s41467-022-28963-9',
 '10.1038/s41467-022-29118-6',
 '10.1038/s41598-022-08691-2',
 '10.1038/s41467-022-29022-z',
 '10.1038/s41467-022-28988-0',
 '10.1186/s40659-022-00382-6',
 '10.1038/s41467-022-28959-5',
 '10.1038/s41528-022-00147-x',
 '10.1038/s41467-022-28880-x',
 '10.1038/s41699-022-00287-8',
 '10.1038/s41598-022-07853-6',
 '10.1038/s41598-022-07858-1',
 '10.1038/s41467-022-28792-w',
 '10.1038/s41467-022-28793-9',
 '10.1007/s40820-022-00793-w',
 '10.1038/s41598-022-07036-3',
 '10.1007/s40820-022-00806-8',
 '10.1038/s42004-022-00626-2',
 '10.1038/s41528-022-00144-0',
 '10.1038/s42004-022-00628-0',
 '10.1038/s41467-022-28380-y',
 '10.1038/s41467-022-28325-5',
 '10.1038/s41524-021-00693-4',
 '10.1038/s41467-022-28381-x',
 '10.1038/s41467-022-28176-0']

In [21]:
def download_doi(doi,num_list):
    """
    Download the paper according to the DOI
    :param doi: <str> DOI
    """
    with open("doi_" + str(num_list) + '.xml', 'w', encoding='utf-8') as f:
        request_url = 'https://api.elsevier.com/content/article/doi/{}?apiKey={}&httpAccept=text%2Fxml'.format(
            doi, API_KEY)
        print(request_url)
        text = requests.get(request_url).text
        f.write(text)
    return

In [22]:
for i in range(25):
  download_doi(doi[i],num_list[i])

https://api.elsevier.com/content/article/doi/10.1038/s41467-022-28963-9?apiKey=856d163d12a1f149d801ee44e166857e&httpAccept=text%2Fxml
https://api.elsevier.com/content/article/doi/10.1038/s41467-022-29118-6?apiKey=856d163d12a1f149d801ee44e166857e&httpAccept=text%2Fxml
https://api.elsevier.com/content/article/doi/10.1038/s41598-022-08691-2?apiKey=856d163d12a1f149d801ee44e166857e&httpAccept=text%2Fxml
https://api.elsevier.com/content/article/doi/10.1038/s41467-022-29022-z?apiKey=856d163d12a1f149d801ee44e166857e&httpAccept=text%2Fxml
https://api.elsevier.com/content/article/doi/10.1038/s41467-022-28988-0?apiKey=856d163d12a1f149d801ee44e166857e&httpAccept=text%2Fxml
https://api.elsevier.com/content/article/doi/10.1186/s40659-022-00382-6?apiKey=856d163d12a1f149d801ee44e166857e&httpAccept=text%2Fxml
https://api.elsevier.com/content/article/doi/10.1038/s41467-022-28959-5?apiKey=856d163d12a1f149d801ee44e166857e&httpAccept=text%2Fxml
https://api.elsevier.com/content/article/doi/10.1038/s41528-02

In [32]:
https://api.elsevier.com/content/article/doi/10.1016/j.jallcom.2022.163703?apiKey=856d163d12a1f149d801ee44e166857e&httpAccept=text%2Fxml

SyntaxError: ignored

In [ ]:
https://api.elsevier.com/content/article/doi/10.1016/j.jallcom.2022.163703?apiKey=856d163d12a1f149d801ee44e166857e&httpAccept=text%2Fxml&view=FULL

In [ ]:
for id in doi:
  print(id)
  download_doi(id)